You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [12]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [13]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [14]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [15]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [16]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [17]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [18]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [19]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [25]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit]
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [26]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
**Content Plan Document on Artificial Intelligence**

---

**1. Latest Trends in Artificial Intelligence**
- **Generative AI**: The rise of tools like OpenAI's ChatGPT and DALL-E has revolutionized content creation, art generation, and human-computer interaction.
- **AI Ethics**: Discussion surrounding the ethical implications of AI usage, bias in algorithms, and the importance of transparency.
- **AI in Healthcare**: Growth of AI applications in medical diagnosis, personalized medicine, and operational efficiency in hospitals.
- **AI for Automation**: Increasing adoption of AI in various 

- Display the results of your execution as markdown in the notebook.

In [30]:
from IPython.display import Markdown
Markdown(result.raw)

# The Evolution and Impact of Artificial Intelligence: Trends, Key Players, and Ethical Considerations

## I. Introduction

Artificial Intelligence (AI) has rapidly evolved from a niche technological concept to a transformative force across multiple sectors. As AI technologies continue to mature, they are reshaping industries, redefining job roles, and challenging our ethical boundaries. With advancements occurring at breakneck speed, it is critical for individuals and businesses to understand the implications of AI in today’s world. Embracing this knowledge will empower stakeholders to navigate the complexities of AI, seize opportunities for innovation, and safeguard against potential pitfalls.

We find ourselves at a pivotal moment where the intertwining of humans and machines presents both exciting opportunities and significant challenges. From generative AI tools revolutionizing content creation to ethical debates surrounding algorithmic bias, the impact of AI is all-encompassing. In this blog post, we'll delve into the latest trends in AI, highlight the key players driving the technology's evolution, examine the target audience, and reflect on the practical applications across various sectors.

## II. Latest Trends in Artificial Intelligence

### Generative AI: The Future of Content Creation

One of the most groundbreaking trends in AI is the emergence of generative AI models, such as OpenAI's ChatGPT and DALL-E. These innovative tools have revolutionized content creation by allowing users to generate high-quality text and images with unprecedented ease. As businesses and individuals seek to harness the power of generative AI, the potential for creative expression and efficiency is limitless. Nevertheless, the rise of these technologies also beckons discussions around copyright, originality, and the ethical ramifications of automated content generation.

As generative AI technology continues to evolve, it opens the door to unprecedented collaboration between humans and machines. This partnership can lead to novel creative processes, where AI acts as an assistant rather than a replacement. However, for this collaboration to thrive, it is essential to establish guidelines that govern usage and ensure that the benefits of this technology extend beyond a select few.

### AI Ethics: Responsibility in Deployment

As we revel in the possibilities that AI presents, it's equally crucial to address the ethical dilemmas that accompany its deployment. Issues such as algorithmic bias, transparency, and accountability are at the forefront of discussions surrounding AI ethics. Striking a balance between innovation and responsibility is paramount. As organizations develop AI systems, they must prioritize ethical considerations to avoid exacerbating social inequalities and undermining trust in technology. 

Moreover, incorporating diverse perspectives during the design process is essential in creating AI systems that serve all segments of society. As we address ethical concerns, the dialogue around responsible AI development must be inclusive, fostering a well-rounded approach to these technologies.

### AI in Healthcare: Transforming Patient Care

AI's potential to revolutionize healthcare is profound. From diagnosing diseases with remarkable accuracy to offering personalized treatment plans, AI applications in healthcare enhance patient outcomes and operational efficiency. Machine learning algorithms analyze vast datasets, flagging potential health risks before they escalate. Moreover, AI-powered tools streamline hospital operations, reducing wait times and optimizing resource allocation.

However, the integration of AI into healthcare also raises concerns about data privacy and security. Ensuring that patient information remains protected while harnessing AI's capabilities is vital. By prioritizing ethical data usage and establishing robust safeguards, the healthcare industry can fully leverage AI to improve patient care while maintaining public trust.

## III. Key Players Shaping AI Technology

### Overview of Major Companies

As AI technology advances, several organizations stand out as key players worth noting. Leaders like OpenAI, Google AI, IBM Watson, Microsoft, and NVIDIA are shaping the landscape through innovative research and influential deployments. Their contributions have fostered an environment where AI can thrive, and each organization brings its unique strengths to the table.

Recognizing the impact of these key players not only helps us understand the current AI landscape, but also allows us to anticipate upcoming innovations. As these organizations push the boundaries of what AI can achieve, they pave the way for future developments across various sectors.

### Innovations and Contributions of Each Player

- **OpenAI** has pioneered advancements in natural language processing (NLP) and generative models, making strides toward more human-like interactions with machines.
- **Google AI** leverages machine learning to enhance practical applications across its product range, integrating AI seamlessly into everyday user experiences.
- **IBM Watson** focuses on delivering tailored AI solutions for various business sectors, showcasing the versatile potential of AI across industries.
- **Microsoft's** strategic partnerships and significant investments in AI reflect its commitment to embedding AI technologies within productivity software, thus improving user efficiency.
- **NVIDIA** plays a pivotal role in hardware development, supplying powerful GPUs that are essential for training deep learning models, directly influencing AI's capabilities.

## IV. Audience Insights: Who Should Care About AI?

### Breakdown of Target Audience Demographics

Artificial Intelligence intersects with a diverse range of stakeholders, and understanding the audience is crucial for effective communication. Tech professionals are keen to explore the technical side of AI, seeking best practices for implementation. Business leaders, driven by a need to maintain competitive advantages, are eager to learn about practical applications of AI within their operations.

Similarly, students and educators are exploring AI for career development opportunities. By providing accessible information about AI, we empower the next generation to embrace the technology and pursue careers within this expanding field.

### Addressing Pain Points and Interests of Each Group

The general public remains curious about AI's societal implications, job displacement fears, and daily life impacts. For these individuals, understanding the nuances of AI technology can demystify misconceptions, fostering a more informed population that can adapt to the evolving landscape.

Additionally, ethics advocates are concerned about the socio-ethical aspects of AI technologies. By addressing these pain points through meaningful dialogue and education, we can cultivate an inclusive conversation around AI that benefits all stakeholders involved.

## V. Practical Applications of AI in Various Sectors

### AI in Business Operations

AI adoption in business operations is accelerating, with companies leveraging AI-driven data analytics to inform decisions, enhance marketing strategies, and optimize supply chains. As the technology allows for historical data mining and pattern recognition, businesses can streamline their processes, leading to cost reduction and improved overall efficiency.

Looking ahead, the role of AI in business is expected to become even more integral, opening avenues for predictive analytics and automated decision-making. Organizations that embrace AI can position themselves as industry leaders by harnessing its potential to drive growth and innovation.

### AI Enhancements in the Tech Industry

In the tech industry, AI applications range from software development to cybersecurity. Intelligent algorithms help detect anomalies in network security, enhancing threat detection and response times. As innovation continues, we can expect more breakthroughs, empowering developers to create smarter, more resilient systems.

Furthermore, as AI technologies become increasingly integrated into tech products, we can anticipate a new era of user experience tailored to individual preferences. By leveraging AI to enhance functionality, tech companies can provide solutions that adapt to the ever-changing demands of users.

### AI in Education and Learning

In the education sector, AI-powered platforms offer personalized learning experiences, adapting content to the student's individual needs. By providing targeted resources and assessments, AI can foster a more engaging and effective learning environment, bridging gaps and enhancing educational outcomes.

Looking ahead, AI's role in education could expand further, offering real-time feedback and adaptive learning systems that cater to diverse learning styles. By integrating AI into educational frameworks, we can create more inclusive and accessible learning environments that foster success for all students.

## VI. Final Considerations: The Future of AI

### Speculation on Upcoming Trends

As we look to the future, several trends are anticipated to shape the AI landscape further. Innovations in explainable AI—transparency in decision-making processes—and continued advancements in NLP will foster more sophisticated interactions between humans and machines. Moreover, regulatory frameworks are likely to emerge to balance innovation and accountability, ultimately guiding responsible AI development.

The collaboration between AI researchers and policymakers will be crucial in shaping future guidelines. By fostering an open dialogue, we can create a collaborative environment that promotes ethical AI development while harnessing its potential.

### Call for Responsible AI Development

The potential of AI should not overshadow the necessity for responsible development practices. As technology continues to evolve, stakeholders must stay informed, emphasizing ethical considerations that would allow AI to be a force for good in society. A collaborative effort between researchers, developers, policymakers, and the public is vital to ensure that AI serves humanity's best interests.

This commitment to responsibility will not only help to avoid detrimental consequences but also foster an environment in which innovation can flourish sustainably. By prioritizing ethical development, stakeholders can ensure that the advancement of AI enhances human experiences.

## VII. Call to Action

As we continue to navigate this rapidly evolving field, I encourage readers to stay informed about the latest developments and trends in Artificial Intelligence. Engage in conversations, share your experiences, and contribute to dialogues surrounding AI's implications on society. By participating in this discourse, we can collectively shape a future where AI enhances our lives while addressing its ethical challenges.

In conclusion, AI is not just a technological trend; it's a pivotal force that necessitates our attention and understanding. As we continue to explore and advance these technologies, let’s do so with a purposeful focus on impact and ethics. By understanding the influences, implications, and applications of AI, we can better position ourselves to face the future with optimism and responsibility.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).